# Hands-on Lab: Build an Interactive Dashboard with Ploty Dash

In this lab, you will be building a Plotly Dash application for users to perform interactive visual analytics on SpaceX launch data in
real-time.

This dashboard application contains input components such as a dropdown list and a range slider to
interact with a pie chart and a scatter point chart. You will be guided to build this dashboard application via the following tasks:

TASK 1: Add a Launch Site Drop-down Input Component<br>
TASK 2: Add a callback function to render success-pie-chart based on selected site dropdown<br>
TASK 3: Add a Range Slider to Select Payload<br>
TASK 4: Add a callback function to render the success-payload-scatter-chart scatter plot<br>

##### Estimated time needed: 90 minutes

In [5]:
#installing required packages  
!pip install pandas dash
!pip install wget

In [6]:
import dash
from dash import dcc
from dash import html, ctx
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px


### Download a skeleton dashboard application and dataset

In [8]:
# download dataset
import wget
wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")

'spacex_launch_dash (9).csv'

In [9]:
# Download a skeleton Dash app to be completed in this lab:
#wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py")

In [10]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [ ]:
 filtered_df = spacex_df[spacex_df['Launch Site'] == value].groupby(['Launch Site', 'class']). \
        size().reset_index(name='class count')
        title = f"Total Success Launches for site {value}"
        fig = px.pie(filtered_df,values='class count', names='class', title=title)
        return fig

In [13]:
app = dash.Dash(__name__)

app.layout=html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                             style={'textAlign':'center', 'color':'#503D36','font-size':40}),
                              # Add a dropdown list to enable Launch Site selection
                              dcc.Dropdown(id='site-dropdown',
                                           options=[
                                               {'label':'All Sites', 'value':'All'},
                                               {'label':'CCAFS LC-40','value':'CCAFS LC-40'},
                                               {'label':'VAFB SLC-4E', 'value':'VAFB SLC-4E'},
                                               {'label':'KSC LC-39A', 'value':'KSC LC-39A'},
                                               {'label':'CCAFS SLC-40', 'value':'CCAFS SLC-40'}
                                           ],
                                           value='ALL',
                                           placeholder="Select a Launch Site here",
                                           searchable=True
                                          ),
                             html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),
                               html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                min=0,
                                max=10000,
                                step=1000,
                                value=[min_payload,max_payload],
                                marks={
                                0: '0 kg',
                                2500: '2500',
                                5000: '5000',
                                7500: '7500',
                                10000: '10000'
                                }),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])
 #TASK 2: Add a callback function to render success-pie-chart based on selected site dropdown

# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie(value):
    filtered_df = spacex_df
    if value == 'All':
        fig = px.pie(filtered_df, values='class', names='Launch Site', title='Total Success Launches By Site')
        return fig

    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == value].groupby(['Launch Site', 'class']). \
        size().reset_index(name='class count')
        title = f"Total Success Launches for site {value}"
        fig = px.pie(filtered_df,values='class count', names='class', title=title)
        return fig

#TASK 3: Add a Range Slider to Select Payload
                              
         

#TASK 4: Add a callback function to render the success-payload-scatter-chart scatter plot    
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id='payload-slider', component_property='value')])
def get_scatter_chart(selected_site, payload_range):
    low, high = payload_range
    if selected_site == 'All':
        df = spacex_df
    else:
        df = spacex_df[spacex_df['Launch Site'] == selected_site]
    df_mask = df[(df['Payload Mass (kg)'] >= low) & (df['Payload Mass (kg)'] <= high)]

    fig = px.scatter(df_mask, x='Payload Mass (kg)', y='class', color='Booster Version Category',
                     title='Correlation between Payload and Success')
    
    return fig           

                            
                              
if __name__ == '__main__':
    app.run_server()

### Finding Insights Visually
Now with the dashboard completed, you should be able to use it to analyze SpaceX launch data, and answer the following questions:

Which site has the largest successful launches? **KSC LC 39A**<br>
Which site has the highest launch success rate?**KSC LC 39A**<br>
Which payload range(s) has the highest launch success rate?**2500-5000KG**<br>
Which payload range(s) has the lowest launch success rate?**7500-10000**<br>
Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest
launch success rate?**FT**<br>

### Plotly Dash Reference
Dropdown (input) component
https://dash.plotly.com/dash-core-components/dropdown

Range slider (input) component
https://dash.plotly.com/dash-core-components/rangeslider

Pie chart (output) component
https://plotly.com/python/pie-charts/

Scatter chart (output) component
https://plotly.com/python/line-and-scatter/s

### Author
Yan Luo

### Other contributor(s)